In [5]:
import os
import pandas as pd


BASE_PATH = os.path.dirname(os.getcwd())

In [8]:
dataset = pd.read_excel(f"{BASE_PATH}/data/cplex_result_model_v1.ods", sheet_name="axt")

In [22]:
for experiment in set(dataset["Experimento"]):
    # Recupera dados apenas do experimento, padronizando
    axt = dataset.loc[dataset["Experimento"] == experiment]\
        .dropna(axis=1).drop(["Experimento", "I"], axis=1).values
    for i in range(0, len(axt)):
        for j in range(0, len(axt[i])):
            if axt[i][j] > 1:
                axt[i][j] = axt[i][j] / 100

    # Cálculo do coeficiente de gini
    part1 = 0
    part2 = 0
    for i in range(0, len(axt)):
        for j in range(0, len(axt[i])):
            part2 = part2 + axt[i][j]
            for k in range(0, len(axt[i])):
                part1 = part1 + abs(axt[i][j] - axt[i][k])

    gini = part1 / (2 * len(axt) * part2)
    print(f"{experiment}: {round(gini, 2)}")

EXP_S2: 0.29
EXP_S9: 0.43
EXP_S10: 0.4
EXP_D1: 0.17
EXP_L3: 0.27
EXP_L2: 0.12
EXP_D2: 0.17
EXP_D6: 0.62
EXP_S1 : 0.19
EXP_L11: 0.36
EXP_L6: 0.76
EXP_D3: 0.31
EXP_S11: 0.32
EXP_D9: 0.35
EXP_L1: 0.15
EXP_D10: 0.22
EXP_D11: 0.25
EXP_S3: 0.4
EXP_S6: 0.88
